# Data Ingestion Code (From RiftCodex)

## Loading Supabase

In [6]:
import os
from dotenv import load_dotenv
from supabase import create_client, Client

# Load environment variables
load_dotenv()

# Access keys
supabase: Client = create_client(
    supabase_url=os.getenv("SUPABASE_URL"),
    supabase_key=os.getenv("SUPABASE_SERVICE_ROLE_KEY")
)

## Set Data Ingestion

In [2]:
import http.client

conn = http.client.HTTPSConnection("api.riftcodex.com")
payload = ''
headers = {
  'Accept': 'application/json'
}
conn.request("GET", "/sets", payload, headers)
res = conn.getresponse()
data = res.read()
data = data.decode("utf-8")

In [ ]:
import json
set_data = json.loads(data)
set_data

{'items': [{'id': '23d4057d-cf01-4133-97ca-e8b641b13ac9',
   'name': 'Origins: Proving Grounds',
   'set_id': 'OGS',
   'label': 'Proving Grounds',
   'card_count': 24,
   'tcgplayer_id': '24439',
   'cardmarket_id': '6289',
   'publish_date': '2025-10-31T00:00:00Z',
   'update_date': ''},
  {'id': '76bc5811-b56c-40ec-9343-c00a0785a9d5',
   'name': 'Spiritforged',
   'set_id': 'SFD',
   'label': 'SFD',
   'card_count': 280,
   'tcgplayer_id': '24519',
   'cardmarket_id': '',
   'publish_date': '2026-02-13T00:00:00Z',
   'update_date': ''},
  {'id': '0fe3414b-c530-4a4d-b86c-168b323e8532',
   'name': 'Origins',
   'set_id': 'OGN',
   'label': 'Origins',
   'card_count': 352,
   'tcgplayer_id': '24344',
   'cardmarket_id': '6286',
   'publish_date': '2025-10-31T00:00:00Z',
   'update_date': ''}],
 'total': 3,
 'page': 1,
 'size': 50,
 'pages': 1}

In [8]:
items = set_data["items"]
items

[{'id': '23d4057d-cf01-4133-97ca-e8b641b13ac9',
  'name': 'Origins: Proving Grounds',
  'set_id': 'OGS',
  'label': 'Proving Grounds',
  'card_count': 24,
  'tcgplayer_id': '24439',
  'cardmarket_id': '6289',
  'publish_date': '2025-10-31T00:00:00Z',
  'update_date': ''},
 {'id': '76bc5811-b56c-40ec-9343-c00a0785a9d5',
  'name': 'Spiritforged',
  'set_id': 'SFD',
  'label': 'SFD',
  'card_count': 280,
  'tcgplayer_id': '24519',
  'cardmarket_id': '',
  'publish_date': '2026-02-13T00:00:00Z',
  'update_date': ''},
 {'id': '0fe3414b-c530-4a4d-b86c-168b323e8532',
  'name': 'Origins',
  'set_id': 'OGN',
  'label': 'Origins',
  'card_count': 352,
  'tcgplayer_id': '24344',
  'cardmarket_id': '6286',
  'publish_date': '2025-10-31T00:00:00Z',
  'update_date': ''}]

In [11]:
filtered_data = [{
    "id": item["id"],
    "set_name": item["name"],
    "set_id": item["set_id"],
    "set_label": item["label"],
    "card_count": item["card_count"],
    "set_publish_date": item["publish_date"]
} for item in items]
response = supabase.table('set').insert(filtered_data).execute()

## Card Data Ingestion

In [21]:
import http.client
import json

conn = http.client.HTTPSConnection("api.riftcodex.com")
headers = {
    'Accept': 'application/json'
}

all_cards = []
page = 1
total_pages = None

while True:
    # Request with page parameter
    conn.request("GET", f"/cards?size=100&sort=public_code&page={page}", '', headers)
    res = conn.getresponse()
    data = res.read().decode("utf-8")
    
    card_data = json.loads(data)
    
    # Add cards from this page
    all_cards.extend(card_data['items'])  # Adjust 'data' to match actual response key
    
    # Check if there are more pages (adjust based on API response structure)
    if total_pages is None:
        total_pages = card_data.get('pages')
    
    print(f"Fetched page {page}/{total_pages}")
    
    # Break if last page
    if page >= total_pages or len(card_data['items']) == 0:
        break
    
    page += 1
    conn = http.client.HTTPSConnection("api.riftcodex.com")  # Reconnect for next request

print(f"Total cards fetched: {len(all_cards)}")

Fetched page 1/7
Fetched page 2/7
Fetched page 3/7
Fetched page 4/7
Fetched page 5/7
Fetched page 6/7
Fetched page 7/7
Total cards fetched: 656


In [28]:
all_cards[2]

{'id': 'c67e85ec-8df3-4585-a1c2-d7c2f999dae3',
 'name': 'Chemtech Enforcer',
 'riftbound_id': 'ogn-003-298',
 'tcgplayer_id': '652773',
 'public_code': 'OGN-003/298',
 'collector_number': 3,
 'attributes': {'energy': 2, 'might': 2, 'power': None},
 'classification': {'type': 'Unit',
  'supertype': None,
  'rarity': 'Common',
  'domain': ['Fury']},
 'text': {'rich': "<p>[Assault 2] <em>(+2 :rb_might: while I'm an attacker.)</em><br />When you play me, discard 1.</p>",
  'plain': "[Assault 2] (+2 :rb_might: while I'm an attacker.)When you play me, discard 1."},
 'set': {'set_id': 'OGN', 'label': 'Origins'},
 'media': {'image_url': 'https://cmsassets.rgpub.io/sanity/images/dsfx7636/game_data_live/19dcf211457d9c9c6e9ea0cd32af76c2c92a3160-744x1039.png',
  'artist': 'Six More Vodka',
  'accessibility_text': "Riftbound Unit: Chemtech Enforcer. [Assault 2] (+2 [S] while I'm an attacker.)\nWhen you play me, discard 1."},
 'tags': ['Zaun'],
 'orientation': 'portrait',
 'metadata': {'clean_name':

In [ ]:
filtered_card_data = [{
    "id": card["id"],
    "set_id": card["set"]["set_id"],
    "card_number": card["collector_number"],
    ""
} for card in all_cards]
response = supabase.table('set').insert(filtered_card_data).execute()